1. pandas 패키지 import

In [ ]:
import pandas as pd

2. csv 파일 읽어오기

In [ ]:
df=pd.read_csv('signal_data.csv')

3. Address1(주소1)에 대해 countplot그래프로 만들기

 1 Seaborn을 활용

 2 Address1(주소1)에 대해서 분포를 보여주는 countplot그래프

 3 지역명이 없는 '-'에 해당되는 row(행)을 삭제

In [ ]:
import seaborn as sns

sns.countplot(data=df, x='Address1')
plt.show()

df = df[df['Address1'] != '-']

4. 실주행시간과 평균시속의 분포 확인

 1 Time_Driving(실주행시간)과 Speed_Per_Hour(평균시속)을 jointplot 그래프 그리기

 2 Seaborn을 활용

 3 X축에는 Time_Driving(실주행시간)을 표시하고 Y축에는 Speed_Per_Hour(평균시속)을 표시

In [ ]:
sns.jointplot(data=df, x='Time_Driving',y='Speed_Per_Hour')
plt.show()

5. jointplot 그래프에서 발견한 이상치 1개를 삭제

 1 대상 데이터프레임: df

 2 jointplot 그래프를 보고 시속 300 이상되는 이상치를 찾아 해당 행(Row)을 삭제하세요.

 3 전처리 반영 후에 새로운 데이터프레임 변수명 df_temp에 저장하세요.

In [ ]:
df_temp = df[df['Speed_Per_Hour'] < 300]

6. 가이드에 따른 결측치 제거

1 대상 데이터프레임: df_temp

2 결측치를 확인하는 코드를 작성

3 결측치가 있는 행(raw)를 삭제

4 전처리 반영된 결과를 새로운 데이터프레임 변수명 df_na에 저장

In [ ]:
df_temp.isna().sum()
df_na = df_temp.dropna(axis=0)

7. 불필요한 변수 삭제

1 대상 데이터프레임: df_na

2 'Time_Departure', 'Time_Arrival' 2개 컬럼을 삭제

3 전처리 반영된 결과를 새로운 데이터프레임 변수명 df_del에 저장

In [ ]:
df_del = df_na.drop(['Time_Departure','Time_Arrival'], axis=1)

8. 조건에 해당하는 컬럼 원핫인코딩

1 대상 데이터프레임: df_del

2 원-핫 인코딩 대상: object 타입의 전체 컬럼

3 활용 함수: pandas의 get_dummies

4 해당 전처리가 반영된 결과를 데이터프레임 변수 df_preset에 저장

In [ ]:
cols = df_del.select_dtypes('object').columns
df_preset = pd.get_dummies(data=df_del, columns=cols)

9. 훈련, 검증 데이터셋 분리('Time_Driving'을 예측하는 모델)

1 대상 데이터프레임: df_preset

2 훈련 데이터셋 label: y_train, 훈련 데이터셋 Feature: X_train

3 검증 데이터셋 label: y_valid, 검증 데이터셋 Feature: X_valid

4 훈련 데이터셋과 검증데이터셋 비율은 80:20

5 random_state: 42

6 Scikit-learn의 train_test_split 함수를 활용하세요.

In [ ]:
from sklearn.model_selection import train_test_split
x = df_preset.drop('Time_Driving', axis=1)
y = df_preset['Time_Driving']
X_train, X_valid, y_train, y_valid = train_test_split(x,y, test_size=0.2, random_state=42)

10. Decision tree로 머신러닝 모델 만들고 학습하기

1 트리의 최대 깊이: 5로 설정

2 노드를 분할하기 위한 최소한의 샘플 데이터수(min_samples_split): 3로 설정

3 random_state: 120로 설정

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(max_depth=5, min_samples_split=3, random_state=120)
dt.fit(X_train, y_train)

11. 위 Decision tree 모델 성능 평가하기

1 예측 결과의 mae(Mean Absolute Error)를 구하기

2 성능 평가는 검증 데이터셋을 활용

3 10번 문제에서 만든 의사결정나무(decision tree) 모델로 y값을 예측(predict)하* 여 y_pred에 저장

4 검증 정답(y_valid)과 예측값(y_pred)의 mae(Mean Absolute Error)를 구하고 dt_mae 변수에 저장

In [ ]:
from sklearn.metrics import mean_absolute_error
y_pred = dt.predict(X_valid)
dt_mae = mean_absolute_error(y_valid, y_pred)

12. Time_Driving(실주행시간)을 예측하는 딥러닝 모델을 만들기

1 Tensoflow framework를 사용하여 딥러닝 모델 구현

2 히든레이어(hidden layer) 2개이상으로 모델을 구성

3 dropout 비율 0.2로 Dropout 레이어 1개를 추가

4 손실함수는 MSE(Mean Squared Error)를 사용

5 하이퍼파라미터 epochs: 30, batch_size: 16으로 설정

6 각 에포크마다 loss와 metrics 평가하기 위한 데이터로 X_valid, y_valid 사용

7 학습정보는 history 변수에 저장

In [ ]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics='mse')
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_valid, y_valid))

13. 위 딥러닝 모델 성능 평가

1 Matplotlib 라이브러리 활용해서 학습 mse와 검증 mse를 그래프로 표시

2 1개의 그래프에 학습 mse과 검증 mse 2가지를 모두 표시

3 위 2가지 각각의 범례를 'mse', 'val_mse'로 표시

4 그래프의 타이틀은 'Model MSE'로 표시

5 X축에는 'Epochs'라고 표시하고 Y축에는 'MSE'라고 표시

In [ ]:
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.legend(['mse','val_mse'])
plt.title('Model MSE')
plt.xlabel('Epochs')
plt.ylabel('Mse')